# DATA ANOTATION
Este codigo te pone a anotar esquinas

In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageDraw
import os

# Configuración inicial
input_folder = r"data\ok"
output_folder = os.path.join(input_folder, "annotations")
os.makedirs(output_folder, exist_ok=True)

# Variables globales
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
current_index = 0
rect_start = None
rect_end = None
rect_id = None
zoom_level = 1.0
current_image = None
img_tk = None
offset_x = 0
offset_y = 0
pan_start = None

# Funciones principales
def load_image(index):
    if 0 <= index < len(image_files):
        image_path = os.path.join(input_folder, image_files[index])
        image = Image.open(image_path)
        return image
    return None

def save_rectangle(image, start, end, output_path):
    cropped = image.crop((min(start[0], end[0]), min(start[1], end[1]),
                          max(start[0], end[0]), max(start[1], end[1])))
    cropped.save(output_path)

def on_mouse_press(event):
    global rect_start, rect_id, pan_start
    if event.num == 1:  # Left click
        rect_start = (int((event.x - offset_x) / zoom_level), int((event.y - offset_y) / zoom_level))
        rect_id = canvas.create_rectangle(event.x, event.y, event.x, event.y, outline="red")
    elif event.num == 3:  # Right click
        pan_start = (event.x, event.y)

def on_mouse_drag(event):
    global rect_id, offset_x, offset_y, pan_start
    if rect_id and rect_start:
        canvas.coords(rect_id, rect_start[0] * zoom_level + offset_x, rect_start[1] * zoom_level + offset_y, event.x, event.y)
    elif pan_start:
        dx = event.x - pan_start[0]
        dy = event.y - pan_start[1]
        offset_x += dx
        offset_y += dy
        pan_start = (event.x, event.y)
        update_canvas()

def on_mouse_release(event):
    global rect_start, rect_end, rect_id, current_image, pan_start
    if event.num == 1 and rect_start:  # Left click release
        rect_end = (int((event.x - offset_x) / zoom_level), int((event.y - offset_y) / zoom_level))
        if rect_start and rect_end:
            output_path = os.path.join(output_folder, f"annotation_{current_index}_{rect_start[0]}_{rect_start[1]}_{rect_end[0]}_{rect_end[1]}.png")
            save_rectangle(current_image, rect_start, rect_end, output_path)
        rect_start = None
        rect_end = None
        rect_id = None
    elif event.num == 3:  # Right click release
        pan_start = None

import time

last_zoom_time = 0  # Variable global para limitar la frecuencia de zoom
ZOOM_MIN = 0.1      # Zoom mínimo
ZOOM_MAX = 10.0     # Zoom máximo

def on_mouse_wheel(event):
    global zoom_level, offset_x, offset_y, last_zoom_time
    
    # Limitar frecuencia de zoom
    current_time = time.time()
    if current_time - last_zoom_time < 0.05:  # 50 ms entre eventos
        return
    last_zoom_time = current_time

    # Calcular factor de zoom
    factor = 1.1 if event.delta > 0 else 0.9
    new_zoom_level = zoom_level * factor

    # Restringir niveles de zoom
    if not (ZOOM_MIN <= new_zoom_level <= ZOOM_MAX):
        return

    # Ajustar desplazamiento para centrarse en el cursor
    cursor_x = canvas.canvasx(event.x)
    cursor_y = canvas.canvasy(event.y)
    offset_x = cursor_x - factor * (cursor_x - offset_x)
    offset_y = cursor_y - factor * (cursor_y - offset_y)

    # Actualizar nivel de zoom y redibujar
    zoom_level = new_zoom_level
    update_canvas()


def next_image():
    global current_index, current_image, img_tk, zoom_level, offset_x, offset_y
    current_index += 1
    zoom_level = 1.0
    offset_x = 0
    offset_y = 0
    if current_index < len(image_files):
        current_image = load_image(current_index)
        update_canvas()
    else:
        print("No hay más imágenes.")

# def update_canvas():
#     global img_tk
#     if current_image:
#         resized_image = current_image.resize((int(current_image.width * zoom_level), int(current_image.height * zoom_level)))
#         img_tk = ImageTk.PhotoImage(resized_image)
#         canvas.delete("all")
#         canvas.config(scrollregion=(0, 0, resized_image.width + offset_x, resized_image.height + offset_y))
#         canvas.create_image(offset_x, offset_y, anchor=tk.NW, image=img_tk)

def update_canvas():
    global img_tk
    if current_image:
        # Calcular dimensiones escaladas una vez
        scaled_width = int(current_image.width * zoom_level)
        scaled_height = int(current_image.height * zoom_level)

        # Redimensionar solo si el tamaño cambió
        resized_image = current_image.resize((scaled_width, scaled_height))
        img_tk = ImageTk.PhotoImage(resized_image)

        # Redibujar la imagen (manteniendo el scroll)
        canvas.delete("image")  # Borra solo el identificador de la imagen
        canvas.create_image(offset_x, offset_y, anchor=tk.NW, image=img_tk, tags="image")

        # Actualizar región de scroll
        canvas.config(scrollregion=(0, 0, scaled_width, scaled_height))


# Crear la interfaz gráfica
root = tk.Tk()
root.title("Herramienta de Anotación Rápida")

canvas = tk.Canvas(root)
canvas.pack(fill=tk.BOTH, expand=True)

frame_buttons = tk.Frame(root)
frame_buttons.pack()

btn_next = tk.Button(frame_buttons, text="Siguiente Imagen", command=next_image)
btn_next.pack(side=tk.LEFT)

canvas.bind("<ButtonPress-1>", on_mouse_press)
canvas.bind("<B1-Motion>", on_mouse_drag)
canvas.bind("<ButtonRelease-1>", on_mouse_release)
canvas.bind("<MouseWheel>", on_mouse_wheel)
canvas.bind("<ButtonPress-3>", on_mouse_press)
canvas.bind("<B3-Motion>", on_mouse_drag)
canvas.bind("<ButtonRelease-3>", on_mouse_release)

# Cargar la primera imagen
if image_files:
    current_image = load_image(current_index)
    update_canvas()
else:
    print("No se encontraron imágenes en la carpeta especificada.")

root.mainloop()


c:\Users\DanielHP\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (102513208 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\DanielHP\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (99534279 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\DanielHP\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (100316160 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\DanielHP\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (99322949 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\DanielHP\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (99038940 pixels) exceeds limit of 89478485 pixels, 

No hay más imágenes.


# Random sampler
Este otro codigo coge pedacitos random de las imagenes para crear los negativos

In [ ]:
import os
import random
from PIL import Image

# Configuración inicial
input_folder = r"E:\Universidad\ML\ML-Project\data\Generales-Parciales"
output_folder = os.path.join(input_folder, "random_crops")
os.makedirs(output_folder, exist_ok=True)

# Número total de subimágenes y tamaño del recorte
num_total_crops = 5600
crop_size = 200

# Obtener la lista de imágenes
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
num_images = len(image_files)
if num_images == 0:
    raise ValueError("No se encontraron imágenes en la carpeta especificada.")

# Cantidad de recortes por imagen
crops_per_image = num_total_crops // num_images

# Generar recortes aleatorios
def generate_random_crops(image_path, num_crops, crop_size, output_folder):
    with Image.open(image_path) as img:
        width, height = img.size
        for i in range(num_crops):
            if width < crop_size or height < crop_size:
                raise ValueError(f"La imagen {image_path} es más pequeña que el tamaño del recorte ({crop_size}x{crop_size}).")

            left = random.randint(0, width - crop_size)
            top = random.randint(0, height - crop_size)
            right = left + crop_size
            bottom = top + crop_size

            crop = img.crop((left, top, right, bottom))
            crop_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_crop_{i}.png"
            crop.save(os.path.join(output_folder, crop_filename))

# Procesar cada imagen
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    generate_random_crops(image_path, crops_per_image, crop_size, output_folder)

print(f"Se generaron {num_total_crops} recortes aleatorios de {crop_size}x{crop_size} y se guardaron en la carpeta {output_folder}.")


# Model training

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt  # <-- Importar matplotlib

class BorderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.rotation_angles = [90, 180, 270]  # Ángulos para rotaciones adicionales
        
        borders_dir = os.path.join(root_dir, "borders")
        for img_name in os.listdir(borders_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
                img_path = os.path.join(borders_dir, img_name)
                # Imagen original
                self.samples.append((img_path, 0))
                # Agregar rotaciones
                for angle in self.rotation_angles:
                    self.samples.append((img_path, 0, angle))
        
        no_borders_dir = os.path.join(root_dir, "no_borders")
        for img_name in os.listdir(no_borders_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
                img_path = os.path.join(no_borders_dir, img_name)
                self.samples.append((img_path, 1))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        if len(self.samples[idx]) == 2:
            # Imagen sin rotación
            img_path, label = self.samples[idx]
            image = Image.open(img_path).convert('RGB')
        else:
            # Imagen con rotación
            img_path, label, angle = self.samples[idx]
            image = Image.open(img_path).convert('RGB')
            image = image.rotate(angle)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def check_data_directory(data_dir):
    """Verifica que existan las carpetas necesarias con las imágenes"""
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"El directorio {data_dir} no existe")
    
    borders_dir = os.path.join(data_dir, "borders")
    no_borders_dir = os.path.join(data_dir, "no_borders")
    
    if not os.path.exists(borders_dir):
        raise FileNotFoundError(f"No se encuentra la carpeta 'borders' en {data_dir}")
    if not os.path.exists(no_borders_dir):
        raise FileNotFoundError(f"No se encuentra la carpeta 'no_borders' en {data_dir}")
    
    # Verificar que hay imágenes en las carpetas
    valid_extensions = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')
    borders_images = [f for f in os.listdir(borders_dir) if f.lower().endswith(valid_extensions)]
    no_borders_images = [f for f in os.listdir(no_borders_dir) if f.lower().endswith(valid_extensions)]
    
    if not borders_images:
        raise FileNotFoundError(f"No se encontraron imágenes válidas en {borders_dir}")
    if not no_borders_images:
        raise FileNotFoundError(f"No se encontraron imágenes válidas en {no_borders_dir}")
    
    print(f"Encontradas {len(borders_images)} imágenes con bordes")
    print(f"Después del data augmentation: {len(borders_images) * 4} imágenes con bordes")
    print(f"Encontradas {len(no_borders_images)} imágenes sin bordes")

def main():
    # ==============================================================================
    # CONFIGURACIÓN BÁSICA
    # ==============================================================================
    DATA_DIR = os.path.abspath("dataset")
    BATCH_SIZE = 32
    EPOCHS = 10
    LEARNING_RATE = 1e-3
    
    # Verificar la estructura del directorio y las imágenes
    try:
        check_data_directory(DATA_DIR)
    except FileNotFoundError as e:
        print(f"Error al cargar los datos: {str(e)}")
        print("\nAsegúrate de que tu estructura de directorios sea así:")
        print("dataset/")
        print("├── borders/")
        print("│   ├── imagen1.jpg")
        print("│   └── ...")
        print("└── no_borders/")
        print("    ├── imagen1.jpg")
        print("    └── ...")
        return

    # Dispositivo: GPU si está disponible, sino CPU
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Usando dispositivo: {DEVICE}")

    # ==============================================================================
    # TRANSFORMS Y DATASET
    # ==============================================================================
    data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
    ])

    # Usar nuestro dataset personalizado
    full_dataset = BorderDataset(DATA_DIR, transform=data_transforms)
    
    # Separar en train y valid (80% - 20%)
    total_size = len(full_dataset)
    train_size = int(0.8 * total_size)
    val_size = total_size - train_size

    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    # Ajustar num_workers a 0 para evitar bloqueos (sobre todo en Windows)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    print(f"Total de imágenes (incluyendo augmentation): {total_size}")
    print(f"Imágenes de entrenamiento: {train_size}")
    print(f"Imágenes de validación: {val_size}")

    # ==============================================================================
    # DEFINICIÓN DE LA RED NEURONAL CONVOLUCIONAL
    # ==============================================================================
    class BorderDetectionCNN(nn.Module):
        def __init__(self, num_classes=2):
            super(BorderDetectionCNN, self).__init__()
            
            # Capas convolucionales
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
            self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
            self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
            
            # BatchNorm
            self.bn1 = nn.BatchNorm2d(32)
            self.bn2 = nn.BatchNorm2d(64)
            self.bn3 = nn.BatchNorm2d(128)
            self.bn4 = nn.BatchNorm2d(256)
            
            # Dropout para regularización
            self.dropout = nn.Dropout(0.5)
            
            # MaxPool 2x2
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
            
            # Clasificador final
            # 224 -> 112 -> 56 -> 28 -> 14 (reducción cada pool)
            self.fc1 = nn.Linear(256 * 14 * 14, 512)
            self.fc2 = nn.Linear(512, num_classes)
            
            self.relu = nn.ReLU()

        def forward(self, x):
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
            x = self.pool(x)

            x = self.conv2(x)
            x = self.bn2(x)
            x = self.relu(x)
            x = self.pool(x)

            x = self.conv3(x)
            x = self.bn3(x)
            x = self.relu(x)
            x = self.pool(x)

            x = self.conv4(x)
            x = self.bn4(x)
            x = self.relu(x)
            x = self.pool(x)

            x = x.view(x.size(0), -1)  # Flatten

            x = self.fc1(x)
            x = self.relu(x)
            x = self.dropout(x)
            x = self.fc2(x)
            return x

    # Instanciar el modelo y moverlo a GPU/CPU
    model = BorderDetectionCNN(num_classes=2).to(DEVICE)

    # Definir función de pérdida y optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Scheduler para ajuste dinámico del LR (opcional)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

    # ==============================================================================
    # ENTRENAMIENTO
    # ==============================================================================

    # -- Inicializar listas para graficar
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    
    # -- Configuración de matplotlib en modo interactivo
    plt.ion()  
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    def train_one_epoch(epoch):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            if (i + 1) % 10 == 0:
                print(f'  Época [{epoch+1}/{EPOCHS}], Batch [{i+1}/{len(train_loader)}], '
                      f'Loss: {loss.item():.4f}')
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100.0 * correct / total
        return epoch_loss, epoch_acc

    def validate_one_epoch():
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss = running_loss / len(val_loader)
        val_acc = 100.0 * correct / total
        return val_loss, val_acc

    best_val_loss = float('inf')
    start_time = time.time()
    
    for epoch in range(EPOCHS):
        print(f"==> Época {epoch+1}/{EPOCHS} <==")
        train_loss, train_acc = train_one_epoch(epoch)
        val_loss, val_acc = validate_one_epoch()
        
        # Guardar resultados
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        # Actualizar learning rate con el scheduler (si se usa)
        scheduler.step(val_loss)
        
        # Guardar el mejor modelo (según val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "mejor_modelo_bordes.pth")

        # Mostrar métricas de la época
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")
        print("-" * 60)

        # -------------------------------------------------------------
        #       Actualización de las gráficas (cada época)
        # -------------------------------------------------------------
        ax1.clear()
        ax1.plot(range(1, epoch+2), train_losses, label='Train Loss')
        ax1.plot(range(1, epoch+2), val_losses, label='Val Loss')
        ax1.set_title('Loss durante el entrenamiento')
        ax1.set_xlabel('Época')
        ax1.set_ylabel('Loss')
        ax1.legend()

        ax2.clear()
        ax2.plot(range(1, epoch+2), train_accuracies, label='Train Acc')
        ax2.plot(range(1, epoch+2), val_accuracies, label='Val Acc')
        ax2.set_title('Accuracy durante el entrenamiento')
        ax2.set_xlabel('Época')
        ax2.set_ylabel('Accuracy (%)')
        ax2.legend()

        plt.tight_layout()
        plt.pause(0.01)  # Pausa pequeña para actualizar la gráfica

    total_time = time.time() - start_time
    print(f"Entrenamiento finalizado en {total_time:.2f} segundos.")
    print(f"Mejor modelo guardado como 'mejor_modelo_bordes.pth'")

    # Detenemos el modo interactivo para que la ventana no se cierre inmediatamente
    plt.ioff()
    # Mantener la gráfica al finalizar
    plt.show()

if __name__ == "__main__":
    main()


# Heatmap

Para cambiar el tamannio de los cuadraditos cambia:
> WINDOW_SIZE = height // 75

Ademas ten en cuenta que yo estoy haciendole el heatmap a una imagen en la misma carpeta llamada habana2.jpg

In [ ]:
import os
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn

# Definir el modelo previamente entrenado
class BorderDetectionCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(BorderDetectionCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.pool(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Cargar el modelo entrenado
model_path = "mejor_modelo_bordes.pth"
model = BorderDetectionCNN(num_classes=2)
model.load_state_dict(torch.load(model_path, map_location="cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Leer la imagen
image_path = "habana2.jpg"
original_image = Image.open(image_path).convert("RGB")

# Convertir a escala de grises
img_array = np.array(original_image.convert("L"))

height, width = img_array.shape
WINDOW_SIZE = height // 75
if WINDOW_SIZE < 2:
    WINDOW_SIZE = 2

STEP_SIZE = WINDOW_SIZE // 2
if STEP_SIZE < 1:
    STEP_SIZE = 1

print(f"WINDOW_SIZE = {WINDOW_SIZE}, STEP_SIZE = {STEP_SIZE}")

# Crear heatmap y mapa de conteo
heatmap = np.zeros((height, width), dtype=np.float32)
count_map = np.zeros((height, width), dtype=np.float32)

# Transformación para el modelo
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Pasar por cada ventana deslizante
for y in range(0, height - WINDOW_SIZE, STEP_SIZE):
    for x in range(0, width - WINDOW_SIZE, STEP_SIZE):
        patch = img_array[y : y + WINDOW_SIZE, x : x + WINDOW_SIZE]
        
        # Convertir a 3 canales
        patch_3ch = np.stack([patch, patch, patch], axis=2)
        
        # Transformar y pasar por el modelo
        patch_tensor = transform(patch_3ch).unsqueeze(0).to(device)
        
        with torch.no_grad():
            logits = model(patch_tensor)
            probs = F.softmax(logits, dim=1)

        # Probabilidad de borde (clase 0)
        prob_border = 1-probs[0, 0].item()

        heatmap[y : y + WINDOW_SIZE, x : x + WINDOW_SIZE] += prob_border
        count_map[y : y + WINDOW_SIZE, x : x + WINDOW_SIZE] += 1

# Normalizar heatmap
count_map = np.maximum(count_map, 1e-5)
heatmap /= count_map
heatmap_normalized = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)

# Crear mapa de colores y superposición
base_img_color = cv2.cvtColor(img_array, cv2.COLOR_GRAY2BGR)
cmap = cm.get_cmap('jet')
heatmap_color = cmap(heatmap_normalized)[..., :3]  # Eliminar canal alpha
heatmap_color = (heatmap_color * 255).astype(np.uint8)

# Superponer el heatmap con la imagen base
alpha = 0.5
overlay = cv2.addWeighted(base_img_color, 1.0 - alpha, heatmap_color, alpha, 0)

# Guardar resultado
output_path = "heatmap_output.png"
cv2.imwrite(output_path, overlay)
print("Heatmap guardado en:", output_path)

# Mostrar resultado
plt.figure(figsize=(10, 10))
plt.imshow(overlay[..., ::-1])  # Cambiar BGR a RGB para matplotlib
plt.title("Heatmap de Bordes")
plt.axis("off")
plt.show()
